In [1]:
import mcstasscript as ms
my_configurator = ms.Configurator()
my_configurator.set_mcrun_path("/usr/bin/")
my_configurator.set_mcstas_path("/usr/share/mcstas/3.4/")

In [2]:
instrument = ms.McStas_instr("my_instrument")

In [3]:
instrument.available_components()

Here are the available component categories:
 contrib
 misc
 monitors
 obsolete
 optics
 samples
 sources
 union
Call available_components(category_name) to display


In [4]:
source = instrument.add_component("source", "Source_simple")
source.show_parameters()

 ___ Help Source_simple _____________________________________________________________
|optional parameter|required parameter|default value|user specified value|
radius = 0.1 [m] // Radius of circle in (x,y,0) plane where neutrons are 
                    generated. 
yheight = 0.0 [m] // Height of rectangle in (x,y,0) plane where neutrons are 
                     generated. 
xwidth = 0.0 [m] // Width of rectangle in (x,y,0) plane where neutrons are 
                    generated. 
dist = 0.0 [m] // Distance to target along z axis.
focus_xw = 0.045 [m] // Width of target
focus_yh = 0.12 [m] // Height of target
E0 = 0.0 [meV] // Mean energy of neutrons.
dE = 0.0 [meV] // Energy half spread of neutrons (flat or gaussian sigma).
lambda0 = 0.0 [AA] // Mean wavelength of neutrons.
dlambda = 0.0 [AA] // Wavelength half spread of neutrons.
flux = 1.0 [1/(s*cm**2*st*energy unit)] // flux per energy unit, Angs or meV if 
                                           flux=0, the source emits 1 in 4*

In [5]:
source.xwidth = 0.03
source.yheight = 0.03
source.lambda0 = 3
source.dlambda = 2.2
source.dist = 5
source.focus_xw = 0.01
source.focus_yh = 0.01
print(source)

COMPONENT source = Source_simple(
  yheight = 0.03, // [m]
  xwidth = 0.03, // [m]
  dist = 5, // [m]
  focus_xw = 0.01, // [m]
  focus_yh = 0.01, // [m]
  lambda0 = 3, // [AA]
  dlambda = 2.2 // [AA]
)
AT (0, 0, 0) ABSOLUTE


In [6]:
wavelength = instrument.add_parameter("wavelength", value=3, comment="Wavelength in AA")
source.lambda0 = wavelength
source.dlambda = "0.1*wavelength"
print(source)

COMPONENT source = Source_simple(
  yheight = 0.03, // [m]
  xwidth = 0.03, // [m]
  dist = 5, // [m]
  focus_xw = 0.01, // [m]
  focus_yh = 0.01, // [m]
  lambda0 = wavelength, // [AA]
  dlambda = 0.1*wavelength // [AA]
)
AT (0, 0, 0) ABSOLUTE


In [7]:
sample = instrument.add_component("sans_sample", "Sans_spheres")

In [8]:
sample.set_AT(5, RELATIVE=source)
sample.set_parameters(R=200, xwidth=0.01, yheight=0.01, zdepth=0.01,
                      target_index=1, focus_xw=0.5, focus_yh=0.5)
print(sample)


COMPONENT sans_sample = Sans_spheres(
  R = 200, // [AA]
  xwidth = 0.01, // [m]
  yheight = 0.01, // [m]
  zdepth = 0.01, // [m]
  target_index = 1, // [1]
  focus_xw = 0.5, // [m]
  focus_yh = 0.5 // [m]
)
AT (0, 0, 5) RELATIVE source


In [9]:
PSD = instrument.add_component("PSD", "PSD_monitor")
PSD.set_AT([0, 0, 5], RELATIVE=sample)
PSD.set_parameters(xwidth=sample.focus_xw, yheight=sample.focus_yh, filename='"PSD.dat"')

In [10]:
# PSD_rad = instrument.add_component("PSD_rad", "PSD_monitor_rad")
# PSD_rad.set_AT([0, 0, 5], RELATIVE=sample)
# PSD_rad.set_parameters(rmax=sample.focus_xw, filename='"PSD_rad.dat"')

In [11]:
instrument.show_parameters()

 wavelength  = 3  // Wavelength in AA


In [12]:
instrument.set_parameters(wavelength=5)
instrument.settings(ncount=2E6)

In [13]:
data = instrument.backengine()

---- Found 5 places in McStas output with keyword 'error'. 

ERROR: syntax error at line 28.
----------------------------------------------------------------------
McStas: 1 Errors encountered during parse of instrument.instr.
INFO: call to /usr/share/mcstas/3.4/bin/mcstas failed with Command '/usr/share/mcstas/3.4/bin/mcstas -t -o ./instrument.c instrument.instr' returned non-zero exit status 1.
Traceback (most recent call last):
  File "/usr/share/mcstas/3.4/tools/Python/mcrun/mcrun.py", line 562, in <module>
    main()
  File "/usr/share/mcstas/3.4/tools/Python/mcrun/mcrun.py", line 476, in main
    mcstas.prepare(options)
  File "/usr/share/mcstas/3.4/tools/Python/mcrun/mccode.py", line 143, in prepare
    Process(mccode_bin_abspath).run(['-t', '-o', self.cpath, self.path])
  File "/usr/share/mcstas/3.4/tools/Python/mcrun/mccode.py", line 71, in run
----------------------------------------------------------------------
    raise CalledProcessError(retcode, process.args,
-----------

/home/tbvanderwoude/anaconda3/envs/bep/lib/python3.12/site-packages/mcstasscript/helper/managed_mcrun.py:299: UserWarning: Simulation did not create data folder, most likely failed.
  warnings.warn("Simulation did not create data folder, most likely failed.")
/home/tbvanderwoude/anaconda3/envs/bep/lib/python3.12/site-packages/mcstasscript/helper/managed_mcrun.py:327: UserWarning: No data available to load.
  warnings.warn("No data available to load.")


In [14]:
ms.make_sub_plot(data, log=True)


TypeError: 'NoneType' object is not iterable

In [15]:
print(data)

None


In [16]:
PSD_data = ms.name_search("PSD", data)

In [17]:
PSD_data.Intensity

array([[8.11969647e-43, 8.21882907e-43, 8.48179631e-43, ...,
        9.53070180e-43, 8.24189591e-43, 8.93108899e-43],
       [8.81641215e-43, 9.25223839e-43, 9.43909990e-43, ...,
        1.08742963e-42, 8.77122699e-43, 1.10987005e-42],
       [1.00882523e-42, 7.56418002e-43, 1.03399732e-42, ...,
        7.41707628e-43, 7.83116667e-43, 1.11819159e-42],
       ...,
       [8.17395623e-43, 1.18103175e-42, 7.73142658e-43, ...,
        6.48236756e-43, 9.59918198e-43, 1.16455872e-42],
       [9.18100097e-43, 8.90268958e-43, 5.75340327e-43, ...,
        8.89081706e-43, 1.25598338e-42, 7.92465345e-43],
       [1.08105059e-42, 5.54921414e-43, 8.25151272e-43, ...,
        8.90904808e-43, 8.19936571e-43, 9.17335588e-43]])

In [18]:
intensity_xy = PSD_data.Intensity

In [19]:
info_dict = PSD_data.metadata.info
for field, info in info_dict.items():
    print(field, ":", info)

Date : Tue Apr 23 18:54:39 2024 (1713891279)
type : array_2d(90, 90)
Source : first_instrument (first_instrument.instr)
component : PSD
position : 0 0 10
title : PSD monitor
Ncount : 2000000
filename : PSD.dat
statistics : X0=-0.00924956; dX=3.31325; Y0=0.0393382; dY=3.32335;
signal : Min=3.02054e-43; Max=1.92843e-38; Mean=3.40062e-40;
values : 2.7545e-36 2.69379e-38 1.99994e+06
xvar : X
yvar : Y
xlabel : X position [cm]
ylabel : Y position [cm]
zvar : I
zlabel : Signal per bin
xylimits : -25 25 -25 25
variables : I I_err N
Parameters : {'wavelength': 5.0}
